<a href="https://colab.research.google.com/github/tj-mas04/senseItBot/blob/main/senseItBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import nltk
from nltk.corpus import movie_reviews
import random

# Download necessary NLTK data
nltk.download('movie_reviews')

# Load the movie reviews dataset
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

random.shuffle(documents)

# Extract features and labels
texts = [" ".join(doc) for doc, category in documents]
labels = [category for doc, category in documents]

# Load pre-trained BERT model and tokenizer once
MODEL_NAME = 'nlptown/bert-base-multilingual-uncased-sentiment'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME)
model.eval()  # Set model to evaluation mode

# Function to predict sentiment
def predict_sentiment(text):
    try:
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding='longest', max_length=512)
        with torch.no_grad():  # Disable gradient calculations for efficiency
            outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        sentiment = torch.argmax(probs, dim=-1).item()

        # Mapping the result to a sentiment label
        sentiment_mapping = {
            0: 'very negative',
            1: 'negative',
            2: 'neutral',
            3: 'positive',
            4: 'very positive'
        }
        return sentiment_mapping.get(sentiment, 'neutral')

    except Exception as e:
        print(f"Error in sentiment analysis: {e}")
        return "neutral"

# Function to generate response based on sentiment
def generate_response(sentiment):
    responses = {
        'very positive': [
            "That's wonderful to hear! What made your day so great?",
            "I'm so happy to hear that! Anything special you'd like to share?",
            "That's awesome! I'm glad things are going well for you.",
            "Wow, that sounds amazing! Tell me more about it!",
            "Great news! What's been the highlight of your day?"
        ],
        'positive': [
            "That sounds positive! What else is going on?",
            "Glad to hear that! Anything else you'd like to talk about?",
            "That's great! Do you have any other good news?",
            "I'm happy to hear that! What's been making you smile lately?",
            "Sounds good! Anything exciting coming up?"
        ],
        'neutral': [
            "That's neutral. Is there anything specific on your mind?",
            "I see. Do you want to talk more about it?",
            "Alright. Anything else you'd like to discuss?",
            "Okay. Is there something you're looking forward to?",
            "Got it. How's your day going overall?"
        ],
        'negative': [
            "I'm sorry to hear that. Do you want to talk about what's bothering you?",
            "That sounds tough. What's been going on?",
            "I'm here for you. Do you want to share more about it?",
            "That sounds difficult. Is there any way I can help?",
            "I'm sorry you're feeling this way. What happened?"
        ],
        'very negative': [
            "That's very negative. I'm really sorry to hear that. Do you need someone to talk to?",
            "I'm really sorry to hear that. What's been happening?",
            "That sounds really difficult. Do you want to tell me more about it?",
            "I'm here for you in this tough time. What can I do to support you?",
            "That's really hard to hear. How can I help?"
        ]
    }

    return random.choice(responses.get(sentiment, ["I'm here to chat with you. What's on your mind?"]))

# Chatbot function
def chatbot():
    print("Hello! I am a chatbot. Type 'quit' to exit.")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == 'quit':
            print("Goodbye! Have a great day!")
            break
        sentiment = predict_sentiment(user_input)
        response = generate_response(sentiment)
        print(f"Chatbot (Sentiment: {sentiment}): {response}")

# Run the chatbot
if __name__ == "__main__":
    chatbot()


[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

Hello! I am a chatbot. Type 'quit' to exit.
You: hey 
Chatbot (Sentiment: very positive): I'm so happy to hear that! Anything special you'd like to share?
You: whhat are you
Chatbot (Sentiment: very negative): I'm here for you in this tough time. What can I do to support you?
You: Im feeling low... Need some help
Chatbot (Sentiment: negative): I'm sorry you're feeling this way. What happened?
You: it's just a headache
Chatbot (Sentiment: negative): I'm here for you. Do you want to share more about it?
You: it hurts more but i happy
Chatbot (Sentiment: positive): That sounds positive! What else is going on?
You: im also very nervous about my grades
Chatbot (Sentiment: negative): That sounds tough. What's been going on?
You: quit
Goodbye! Have a great day!
